In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [36]:
name = 'bank'
# df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/defaultCredit/data.csv')
df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/{}/data.csv'.format(name))
# target_clm = 'default.payment.next.month'
target_clm = 'y'
#get numerical columns
dtyp_df = pd.read_csv('/media/6TB_Volume/DataRepo/small_datasets/{}/fold0/data_types.csv'.format(name))
num_types = dtyp_df[dtyp_df.type.isin(['pos','count', 'real'])].index
num_types = pd.DataFrame(columns=list(df.columns[num_types]))
cat_types = dtyp_df[~dtyp_df.type.isin(['pos','count', 'real'])].index
num_categories = dtyp_df[~dtyp_df.type.isin(['pos','count', 'real'])].dim
cat_types = df.columns[cat_types]
cat_types = pd.DataFrame(columns=cat_types)
cat_types.loc[0] = num_categories.values
cat_types = cat_types.drop([target_clm], axis=1)
# factorize the categorical columns
df[cat_types.columns] = df[cat_types.columns].apply(lambda x: pd.factorize(x)[0])
#scale numerical columns
scaler = StandardScaler()
# first we need to detect numerical clms and then scale, this is incorrect
df[num_types.columns] = scaler.fit_transform(df[num_types.columns])
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,1.533034,0,0,0,0,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
1,1.628993,1,0,1,1,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
2,-0.290186,1,0,1,0,1,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
3,-0.002309,2,0,2,0,0,0,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0
4,1.533034,1,0,1,0,0,1,0,0,0,...,-0.565922,0.195414,-0.349494,0,0.648092,0.722722,0.886447,0.71246,0.33168,0


In [38]:
num_types.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/numerical_clms.csv'.format(name), \
     index=False)
cat_types.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/categorical_clms.csv'.format(name), \
     index=False)

In [40]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=142)
df_train, df_valid = train_test_split(df_train, test_size=0.1, random_state=142)
df_train.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/train/data.csv'.format(name), index=False)
df_valid.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/valid/data.csv'.format(name), index=False)
df_test.to_csv('/home/vineeth/Documents/GitWorkSpace/PytorchRecipes/SimpleMLP/Dataset/{}/fold0/test/data.csv'.format(name), index=False)

In [103]:
cat_types[len(cat_types.index)] = num_categories
# num_categories = num_categories.to_frame(index=False).T

In [114]:
num_categories = num_categories.to_frame().T

<h1> Preparing Large Synthetic Datasert </h1>
<li> Create a large synthetic dataset </li>
<li> Split dataset into training, test and validation </li>
<li> Split train, test and validation into multiple parquet files </li>


In [3]:
from sklearn.datasets import make_classification

In [4]:
t_samples = 5000000
feats = 150
X, y = make_classification(
    # same as the previous section
    n_samples=t_samples, n_features=feats, n_informative=100, n_classes=2, 
    # flip_y - high value to add more noise
    flip_y=0.1, 
    # class_sep - low value to reduce space between classes
    class_sep=0.5
)
df = pd.DataFrame(X, columns=['feature_{}'.format(i) for i in range(feats)])
df['label'] = y
df.to_csv('/media/6TB_Volume/DataRepo/LargeSyntheticData/data.csv')

In [5]:
chk = pd.read_csv('/media/6TB_Volume/DataRepo/LargeSyntheticData/data.csv', chunksize=5, index_col=0)
# df = next(chk)

In [6]:
num_features = pd.DataFrame(columns=list(df.columns))
num_features.to_csv('/media/6TB_Volume/DataRepo/LargeSyntheticData/numerical_clms.csv', index=False)

In [3]:
# split the dataset into train, test and valid and split them into parquet files
df = pd.read_csv('/media/6TB_Volume/DataRepo/LargeSyntheticData/data.csv', index_col=0)
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,label
0,-0.324484,7.406514,-0.102909,-1.183279,1.141654,4.123619,-1.896996,1.093310,-6.070587,1.790486,...,-0.657986,-2.212184,1.277470,0.366519,0.509202,0.256471,0.058680,1.041587,2.227189,1
1,0.256281,3.475056,5.964475,-0.699812,1.359575,7.168393,3.661271,0.359714,3.602643,-1.815711,...,1.057984,-0.483040,-3.845169,-0.792224,1.604141,-0.160195,0.131599,1.684299,-1.452773,1
2,-1.172274,3.983185,-1.635864,-0.954257,-1.221992,-1.522706,2.320334,0.685071,-2.404640,1.729606,...,-2.085702,-2.163222,-2.251153,-0.927699,10.948915,-0.186423,0.971816,8.907370,-1.529147,1
3,-0.297441,10.467957,-9.159191,0.621818,-0.073060,-2.072016,4.557587,-0.806728,2.290244,-0.485761,...,5.934013,3.486709,4.153734,-0.555085,-4.489284,0.042981,0.633650,-7.647530,-1.674678,0
4,0.647659,3.667337,-6.661377,-1.367366,-0.096138,-1.344426,10.172251,-1.411733,1.078683,1.569419,...,2.544166,-1.010073,-11.058512,1.510123,1.032110,-0.334906,0.816809,-9.995174,-1.562411,1


In [8]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=142)
df_train, df_valid = train_test_split(df_train, test_size=0.1, random_state=142)

In [5]:
import os
from tqdm import tqdm
def df_to_parquet(df, target_dir, chunk_size=10000, **parquet_wargs):
    """Writes pandas DataFrame to parquet format with pyarrow.

    Args:
        df: DataFrame
        target_dir: local directory where parquet files are written to
        chunk_size: number of rows stored in one chunk of parquet file. Defaults to 1000000.
    """    
    for i in tqdm(range(0, len(df), chunk_size)):
        slc = df.iloc[i : i + chunk_size]
        chunk = int(i/chunk_size)
        fname = os.path.join(target_dir, f"part_{chunk:04d}.parquet")
        slc.to_parquet(fname, engine="pyarrow", **parquet_wargs)
target_dir = '/media/6TB_Volume/DataRepo/LargeSyntheticData/tmp'
df_to_parquet(df, target_dir)

100%|██████████| 500/500 [01:10<00:00,  7.08it/s]


In [2]:
pd.read_parquet('/media/6TB_Volume/DataRepo/LargeSyntheticData/tmp')

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,label
0,-0.324484,7.406514,-0.102909,-1.183279,1.141654,4.123619,-1.896996,1.093310,-6.070587,1.790486,...,-0.657986,-2.212184,1.277470,0.366519,0.509202,0.256471,0.058680,1.041587,2.227189,1
1,0.256281,3.475056,5.964475,-0.699812,1.359575,7.168393,3.661271,0.359714,3.602643,-1.815711,...,1.057984,-0.483040,-3.845169,-0.792224,1.604141,-0.160195,0.131599,1.684299,-1.452773,1
2,-1.172274,3.983185,-1.635864,-0.954257,-1.221992,-1.522706,2.320334,0.685071,-2.404640,1.729606,...,-2.085702,-2.163222,-2.251153,-0.927699,10.948915,-0.186423,0.971816,8.907370,-1.529147,1
3,-0.297441,10.467957,-9.159191,0.621818,-0.073060,-2.072016,4.557587,-0.806728,2.290244,-0.485761,...,5.934013,3.486709,4.153734,-0.555085,-4.489284,0.042981,0.633650,-7.647530,-1.674678,0
4,0.647659,3.667337,-6.661377,-1.367366,-0.096138,-1.344426,10.172251,-1.411733,1.078683,1.569419,...,2.544166,-1.010073,-11.058512,1.510123,1.032110,-0.334906,0.816809,-9.995174,-1.562411,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,-1.413362,0.461580,7.865335,0.131137,1.344820,14.233202,8.117503,-0.915022,-8.358312,0.879691,...,0.234820,-4.024888,-3.060994,0.216188,-1.920941,-0.584419,1.136691,2.533543,-1.244863,1
4999996,-0.091731,4.250108,0.056249,1.179544,-0.385292,0.283680,-6.871068,1.332226,1.356669,-0.972156,...,4.884851,-7.704443,14.468213,0.125388,2.590235,-1.352407,-1.795760,-1.308436,1.099278,1
4999997,-0.276066,-4.391074,8.501095,0.325929,0.639173,-3.735911,-7.734127,1.152391,-6.586978,-0.003637,...,10.009014,-4.561117,7.727924,0.074577,5.724877,0.711676,1.040399,-3.319014,-1.935211,1
4999998,0.101866,-4.738046,2.118930,1.833584,0.279612,8.251644,10.891071,1.288531,6.425323,-0.435636,...,2.208643,-0.397366,6.147679,-1.040513,5.759234,1.079858,1.199316,-6.989284,-0.060882,1
